In [1]:
from operator import itemgetter

import numpy as np
import pandas as pd

import xgboost as xgb

import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

import geopy.distance

In [2]:
df_1 = pd.read_csv("data/sets/train_set_mod_cities.csv", sep=',', encoding='utf-8')
df_1["train"] = True
df_1.head()

/home/vladimir/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon,train
0,2.884034,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,NaN,59.844074,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177,True
1,2.775633,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,NaN,59.844074,30.179153,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,59.847,30.177,True
2,3.708368,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5992,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",59.858200,30.229023,df06c1fcd3718a514535ae822785f716,2017-10-03,59.847,30.177,True
3,2.787498,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,NaN,59.844074,30.179153,6c5e5793ebc984fb72875feffff62854,2017-09-09,59.847,30.177,True
4,2.892510,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,NaN,59.844074,30.179153,0576445d74e374c92c0902e612fca356,2017-07-06,59.847,30.177,True


In [3]:
df_2 = pd.read_csv("data/sets/test_set_mod_cities.csv", sep=',', encoding='utf-8')
df_2["train"] = False
df_2.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon,train
0,2.211818,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,5411,2-1 TOLMACHEVA STRIVANTEEVKA141280 RUSRUS,55.967487,37.913681,ff0476dae4b098a7b16aabe93d4268df,2017-08-24,NaN,NaN,False
1,1.331379,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,4111,"SOVETSKIJ, 32IVANTEEVKA141282 RUSRUS",55.971294,37.905186,7cfd9a60282459d4692ecc85b856072e,2017-08-12,NaN,NaN,False
2,2.608004,NaN,NaN,NaN,PUSHKINO,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,5411,"105,KRASOARMEYSKOE SHPUSHKINO141206 RUSRUS",56.016590,37.909100,7e5a532f0029861d8a9c4f0479b9450b,2017-06-17,NaN,NaN,False
3,1.916752,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,5411,"G. IVANTEEVKA, UL.TOLMACHEVA, D.6IVANTEEVKA141...",55.964508,37.937912,2afe7d1bc61b86c449f413bdf2119032,2017-08-12,NaN,NaN,False
4,1.981067,NaN,NaN,NaN,MOSKVA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,5814,5 KOMSOMOLSKAYA SQMOSCOW101000 RUSRUS,55.776802,37.657352,ab4f00601ff1d949afc59ee3f804c79c,2017-04-26,NaN,NaN,False


In [4]:
df = pd.concat([df_1, df_2], axis=0)

In [5]:
df["lat"] = df["lon"] = np.nan

In [6]:
df_atm_features = pd.read_csv('data/atm_features.csv', sep=',', encoding='utf-8',
                                usecols=['terminal_id', 'lat', 'lon'])
df_atm_features.rename(columns={
        'lat': 'atm_address_lat',
        'lon': 'atm_address_lon'
    }, inplace=True)
df_atm_features.head()

,terminal_id,atm_address_lat,atm_address_lon
0,c693dcbafb5e0c1f9a58ac0211d79ed8,42.056451,48.298047
1,3ecc362617966cb7f4563c9e4e89b8a1,42.056451,48.298047
2,cd614b782e35146bacbad19e800e0624,42.056451,48.298047
3,fca99d9df0e01bdebca13824e1db1dfe,42.056451,48.298047
4,fabf56c3e89566e248ea94636cead5cb,42.061501,48.290501


In [7]:
df.drop(labels=['atm_address_lat', 'atm_address_lon'], axis=1, inplace=True)
df = pd.merge(df, df_atm_features, on='terminal_id', how='left')

In [8]:
mask = df.loc[:, ['atm_address_lat', 'atm_address_lon']].notnull().all(axis=1)
coords_atm = df.loc[mask, ['terminal_id', 'atm_address_lat', 'atm_address_lon']].\
    groupby('terminal_id').median().values
df.loc[mask, ["lat", "lon"]] = df.loc[mask, ['atm_address_lat', 'atm_address_lon']].values

In [9]:
mask = df.loc[:, ['pos_address_lat', 'pos_address_lon']].notnull().all(axis=1)
coords_pos = df.loc[mask, ['terminal_id', 'pos_address_lat', 'pos_address_lon']].\
    groupby('terminal_id').median().values
df.loc[mask, ["lat", "lon"]] = df.loc[mask, ['pos_address_lat', 'pos_address_lon']].values

In [10]:
df = df.loc[df[["lat", "lon"]].notnull().all(axis=1)]

In [11]:
def is_inside_area(lat_pred, lon_pred, lat_true, lon_true, radius=0.02):
    return np.power(lat_pred - lat_true, 2) + np.power(lon_pred - lon_true, 2) <= radius ** 2

In [12]:
def get_banks_area(coords, lat, lon, s=0.02):
    lat_l = np.searchsorted(coords[:, 0], lat - s, side='left')
    lat_r = np.searchsorted(coords[:, 0], lat + s, side='right')
    
    mask = is_inside_area(coords[lat_l:lat_r, 0], coords[lat_l:lat_r, 1], lat, lon, radius=s)
    index = lat_l + np.where(mask)
    index = index[0]
    
    return index

In [13]:
def preprocess_places(col):
    lat_name_col, lon_name_col = map(lambda x: x.format(col), ["{}_add_lat", "{}_add_lon"])
    mask = df.loc[:, [lat_name_col, lon_name_col]].notnull().all(axis=1)
    
    df_cust = df.loc[mask, ["customer_id", lat_name_col, lon_name_col]].\
        groupby("customer_id").median()
    df_cust.sort_values(by=[lat_name_col, lon_name_col], axis=0, inplace=True)
    
    coords = df_cust.values
    
    df_term = df.loc[mask, ["terminal_id", "lat", "lon"]].groupby("terminal_id").median()
                         
    nb_ngb = [
        len(get_banks_area(coords, lat, lon))
        for term_i, lat, lon in df_term.itertuples()
    ]
    print "NGB is done!"

    nb_atm = [
        len(get_banks_area(coords_atm, lat, lon))
        for term_i, lat, lon in df_term.itertuples()
    ]
    print "ATM is done!"

    nb_pos = [
        len(get_banks_area(coords_pos, lat, lon))
        for term_i, lat, lon in df_term.itertuples()
    ]
    print "POS is done!"
    
    df_term["nb_ngb"] = nb_ngb
    df_term["nb_atm"] = nb_atm
    df_term["nb_pos"] = nb_pos
                         
    return df_term

In [14]:
df_cust_home = preprocess_places('home')
df_cust_home.columns = map(lambda x: x+'_home', df_cust_home.columns)
df_cust_home.reset_index(inplace=True)
print df_cust_home.max()
print df_cust_home.shape[0]
df_cust_home.head()

NGB is done!
ATM is done!
POS is done!
terminal_id    ffffc67647bb91f4a31ebc06caacb6f8
lat_home                                 71.581
lon_home                                159.893
nb_ngb_home                                 123
nb_atm_home                                  57
nb_pos_home                                 803
dtype: object
190760


,terminal_id,lat_home,lon_home,nb_ngb_home,nb_atm_home,nb_pos_home
0,0001834f2a236d989a5195bb4e44c247,55.164860,61.431217,5,0,0
1,00026bc30acf76a979c962808852e1c3,55.915054,37.820812,3,0,0
2,000298d8a61b4f7b0fe5c7fd3d8c4c9c,55.598412,37.538109,30,0,260
3,0002c0a947ed793fa8d8fe5672841685,55.770638,37.596565,58,45,48
4,0003600c88e1ed202705acad8a777c6d,55.702103,37.547997,27,0,156


In [15]:
df_cust_work = preprocess_places('work')
df_cust_work.columns = map(lambda x: x+'_work', df_cust_work.columns)
df_cust_work.reset_index(inplace=True)
print df_cust_work.max()
print df_cust_work.shape[0]
df_cust_work.head()

NGB is done!
ATM is done!
POS is done!
terminal_id    ffffc67647bb91f4a31ebc06caacb6f8
lat_work                                 71.581
lon_work                                159.893
nb_ngb_work                                 232
nb_atm_work                                  57
nb_pos_work                                 803
dtype: object
135983


,terminal_id,lat_work,lon_work,nb_ngb_work,nb_atm_work,nb_pos_work
0,00026bc30acf76a979c962808852e1c3,55.915054,37.820812,0,0,0
1,0002c0a947ed793fa8d8fe5672841685,55.770638,37.596565,171,45,48
2,0003e33221153137a3995473acc26205,55.905056,37.405499,4,0,0
3,000430f771792625778b695c3dce5f2c,55.038857,82.961327,5,0,0
4,00048e110f2e31b91780943f1890be9a,50.606728,36.677357,0,0,0


In [16]:
df_cust = pd.merge(
    df_cust_home.loc[:, ['terminal_id', 'nb_ngb_home', 'nb_atm_home', 'nb_pos_home']],
    df_cust_work.loc[:, ['terminal_id', 'nb_ngb_work']],
    on='terminal_id', how='left'
)
df_cust.rename(columns={
        'nb_atm_home': 'nb_atm_term',
        'nb_pos_home': 'nb_pos_term'
    }, inplace=True)
df_cust.fillna(0, inplace=True)
df_cust.loc[:, ['nb_ngb_home', 'nb_atm_term', 'nb_pos_term', 'nb_ngb_work']] = \
    df_cust.loc[:, ['nb_ngb_home', 'nb_atm_term', 'nb_pos_term', 'nb_ngb_work']].astype(int).values
df_cust.head()

,terminal_id,nb_ngb_home,nb_atm_term,nb_pos_term,nb_ngb_work
0,0001834f2a236d989a5195bb4e44c247,5,0,0,0
1,00026bc30acf76a979c962808852e1c3,3,0,0,0
2,000298d8a61b4f7b0fe5c7fd3d8c4c9c,30,0,260,0
3,0002c0a947ed793fa8d8fe5672841685,58,45,48,171
4,0003600c88e1ed202705acad8a777c6d,27,0,156,0


In [17]:
df_cust.to_csv('data/term_features.csv', sep=',', encoding='utf-8', index=False)